In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('assets/gapminder.csv', index_col='Year', thousands=',')
print(data.dtypes)
data.head()

Country        object
life          float64
population    float64
income        float64
region         object
dtype: object


,Country,life,population,income,region
Year,,,,,
1800,Afghanistan,28.211000,3280000.0,603.0,South Asia
1801,Afghanistan,28.200753,NaN,603.0,South Asia
1802,Afghanistan,28.190507,NaN,603.0,South Asia
1803,Afghanistan,28.180260,NaN,603.0,South Asia
1804,Afghanistan,28.170013,NaN,603.0,South Asia


In [3]:
# Get the 2010 slice
data.loc[2010].head()

,Country,life,population,income,region
Year,,,,,
2010,Afghanistan,54.8,27962207.0,1637.0,South Asia
2010,Albania,75.2,2901883.0,9374.0,Europe & Central Asia
2010,Algeria,75.9,36036159.0,12494.0,Middle East & North Africa
2010,Andorra,83.7,84419.0,38982.0,Europe & Central Asia
2010,Angola,59.4,21219954.0,7047.0,Sub-Saharan Africa


In [4]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [5]:
from bokeh.plotting import figure
from bokeh.models import HoverTool

YEAR = 2010

p = figure(
    title=str(YEAR), x_axis_label='income', y_axis_label='life',
    x_axis_type='log', x_range=(100, 100000), y_range=(0, 100),
    plot_height=400, plot_width=700,
)
p.circle(x=data.loc[YEAR].income, y=data.loc[YEAR].life)
show(p)

In [6]:
from bokeh.models import ColumnDataSource

YEAR = 2010

source = ColumnDataSource(
    dict(
        x=data.loc[YEAR].income, 
        y=data.loc[YEAR].life, 
        country=data.loc[YEAR].Country,
    )
)

In [7]:
p = figure(
    title=str(YEAR), x_axis_label='income', y_axis_label='life',
    x_axis_type='log', x_range=(100, 100000), y_range=(0, 100),
    plot_height=400, plot_width=700,
    tools=[HoverTool(tooltips='@country', show_arrow=False)]
)
p.circle(
    x='x', 
    y='y', 
    source=source,
    size=10, 
    alpha=0.8
)
show(p)

In [8]:
source = ColumnDataSource(
    dict(
        x=data.loc[YEAR].income, 
        y=data.loc[YEAR].life, 
        country=data.loc[YEAR].Country,
        pop=data.loc[YEAR].population, 

    )
)

In [9]:
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256
Viridis256.reverse()
color_mapper = LinearColorMapper(palette=Viridis256, low=data.population.min(), high=data.population.max())


p = figure(
    title=str(YEAR), x_axis_label='income', y_axis_label='life',
    x_axis_type='log', x_range=(100, 100000), y_range=(0, 100),
    plot_height=400, plot_width=700,
    tools=[HoverTool(tooltips='@country', show_arrow=False)]
)
p.circle(
    x='x', 
    y='y', 
    source=source,
    size=10, 
    alpha=0.7,
    color={'field': 'pop', 'transform': color_mapper}
)
p.add_layout(ColorBar(color_mapper=color_mapper, location=(0, 0), orientation='horizontal'), 'below')
show(p)

In [10]:
from bokeh.models import LinearInterpolator

p = figure(
    title=str(YEAR), x_axis_label='income', y_axis_label='life',
    x_axis_type='log', x_range=(100, 100000), y_range=(0, 100),
    plot_height=400, plot_width=700,
    tools=[HoverTool(tooltips='@country', show_arrow=False)]
)

size_mappper = LinearInterpolator(
    x=[data.population.min(), data.population.max()],
    y=[5, 70]
)

p.circle(
    x='x', 
    y='y', 
    source=source,
    size={'field': 'pop', 'transform': size_mappper}, 
    alpha=0.7,
    color={'field': 'pop', 'transform': color_mapper}
)
p.add_layout(ColorBar(color_mapper=color_mapper, location=(0, 0), orientation='horizontal'), 'below')
show(p)

In [11]:
from ipywidgets import interact
from bokeh.io import push_notebook

p = figure(
    title=str(YEAR), x_axis_label='income', y_axis_label='life',
    x_axis_type='log', x_range=(100, 100000), y_range=(0, 100),
    plot_height=400, plot_width=700,
    tools=[HoverTool(tooltips='@country', show_arrow=False)]
)
p.circle(
    x='x', 
    y='y', 
    source=source,
    size=10, 
    alpha=0.8
)

def update(year):
    new_data = dict(
        x=data.loc[year].income, 
        y=data.loc[year].life, 
        country=data.loc[year].Country,
    )
    source.data = new_data
    p.title.text = str(year)
    push_notebook()

show(p, notebook_handle=True)

In [12]:
interact(update, year=(1800, 2014, 1))

<function __main__.update>

In [22]:
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6

source = ColumnDataSource(
    dict(
        x=data.loc[YEAR].income, 
        y=data.loc[YEAR].life, 
        country=data.loc[YEAR].Country,
        pop=data.loc[YEAR].population, 
        region=data.loc[YEAR].region,

    )
)

p = figure(
    title=str(YEAR), x_axis_label='income', y_axis_label='life',
    x_axis_type='log', x_range=(100, 100000), y_range=(0, 100),
    plot_height=400, plot_width=700,
    tools=[HoverTool(tooltips='@country', show_arrow=False)]
)

size_mappper = LinearInterpolator(
    x=[data.population.min(), data.population.max()],
    y=[5, 70]
)
color_mapper = CategoricalColorMapper(
    factors=list(data.region.unique()),
    palette=Spectral6
)
p.circle(
    x='x', 
    y='y', 
    source=source,
    alpha=0.8,
    size={'field': 'pop', 'transform': size_mappper}, 
    color={'field': 'region', 'transform': color_mapper},
    label='region'
)

p.legend.location = (5, -100)
p.legend.border_line_color = None
p.right.append(p.legend[0])
show(p)